<a href="https://colab.research.google.com/github/ebelleng/EDDA_desafio1/blob/refactoring/Copy_of_Packing_squares_algoritms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packing Squares 


## Definicion de clases y constantes



In [52]:
# Constantes
CANT_SQUARES = 8
BOARD_SIZE = 5

# Largo de los cuadrados a posicionar
SQUARES = [3,2,2,2,1,1,1,1]

In [53]:
class Accion:
  def __init__(self, i, x, y):
    self.i = i
    self.x = x
    self.y = y

  def __str__(self):
    return f'{self.i}: ({self.x}, {self.y})'

  def __repr__(self):
    return self.__str__()

In [75]:
class Estado:
  def __init__(self, i=0, x=0, y=0):
    self.X = [0] * CANT_SQUARES
    self.Y = [0] * CANT_SQUARES
    self.X[i] = x
    self.Y[i] = y

  def isLastState(self):
    return self.X[-1] != 0

  def getCurrent(self):
    return self.X.index(0)

  def checkCollitions(self, x, y, size):
    current = self.getCurrent()
    for i in range(current):
      if Estado.checkCollition(self.X[i], self.Y[i], SQUARES[i], x, y, size):
        return True
    
    return False

  @staticmethod
  def checkCollition(x1, y1, s1, x2, y2, s2):
    '''
    Verifica si los dos cuadrados se solapan
    x, y: posicion
    s: largo del cuadrado
    '''
    if (y1 + s1) <= y2:
      return False
    elif y1 >= (y2 + s2):
      return False
    elif (x1 + s1) <= x2:
      return False
    elif x1 >= (x2 + s2):
      return False
    else:
      return True

  def __str__(self):
    return f'X={self.X}  Y={self.Y}'

## Funciones

In [55]:
def transicion(estado, accion):
  estado_siguiente = Estado(accion.i, accion.x, accion.y)
  
  for j in range(accion.i):
    estado_siguiente.X[j] = estado.X[j]
    estado_siguiente.Y[j] = estado.Y[j]

  return estado_siguiente

In [68]:
def obtener_acciones(state):
  current = state.getCurrent()
  size = SQUARES[current]
  actions = []

  # Se obtienen las celdas validas para que el cuadrado no se salga del tablero
  valid_cells = [(i, j) for i in range(1, BOARD_SIZE - size + 2) for j in range(1, BOARD_SIZE - size + 2)]

  # cell -> dupla que contiene la posicion (cell[0] = X, cell[1]=Y)
  for cell in valid_cells:
    has_collition = state.checkCollitions(cell[0], cell[1], size)

    if not has_collition:
      actions.append(Accion(current, cell[0], cell[1]))
  
  return actions

In [57]:
# Busca el mayor area cuadrada libre en el estado 'state'
def getEvaluation(state):
  max_size = BOARD_SIZE

  while max_size > 0:
    # Se obtienen las celdas validas para que el cuadrado no se salga del tablero
    valid_cells = [(i, j) for i in range(1, BOARD_SIZE - max_size + 2) for j in range(1, BOARD_SIZE - max_size + 2)]

    # cell -> dupla que contiene la posicion (cell[0] = X, cell[1]=Y)
    for cell in valid_cells:
      has_collition = state.checkCollitions(cell[0], cell[1], max_size)

      # El primer cuadrado que no tenga colisiones, sera la mayor area cuadrada
      if not has_collition:
        return max_size

    # Se sigue buscando en una area cuadrada menor
    max_size -= 1
  
  return -1

In [58]:
def depthFirstSearch(initial_state):
  # Lista que se usara como PILA
  states = [initial_state]

  while len(states) != 0:
    current_state = states.pop()

    if current_state.isLastState():
      return current_state

    actions = obtener_acciones(current_state)
    for a in actions:
      new_state = transicion(current_state, a)
      states.append(new_state)

  return None

## Ejemplos y anotaciones

In [73]:
# Ejemplo de como se van calculando las acciones posibles
ss = Estado()

for i in range(CANT_SQUARES):
  print(ss)
  actions = obtener_acciones(ss)
  ss = transicion(ss, actions[0])
  print(actions, end="\n\n")

print(ss)

X=[0, 0, 0, 0, 0, 0, 0, 0]  Y=[0, 0, 0, 0, 0, 0, 0, 0]
[0: (1, 1), 0: (1, 2), 0: (1, 3), 0: (2, 1), 0: (2, 2), 0: (2, 3), 0: (3, 1), 0: (3, 2), 0: (3, 3)]

X=[1, 0, 0, 0, 0, 0, 0, 0]  Y=[1, 0, 0, 0, 0, 0, 0, 0]
[1: (1, 4), 1: (2, 4), 1: (3, 4), 1: (4, 1), 1: (4, 2), 1: (4, 3), 1: (4, 4)]

X=[1, 1, 0, 0, 0, 0, 0, 0]  Y=[1, 4, 0, 0, 0, 0, 0, 0]
[2: (3, 4), 2: (4, 1), 2: (4, 2), 2: (4, 3), 2: (4, 4)]

X=[1, 1, 3, 0, 0, 0, 0, 0]  Y=[1, 4, 4, 0, 0, 0, 0, 0]
[3: (4, 1), 3: (4, 2)]

X=[1, 1, 3, 4, 0, 0, 0, 0]  Y=[1, 4, 4, 1, 0, 0, 0, 0]
[4: (4, 3), 4: (5, 3), 4: (5, 4), 4: (5, 5)]

X=[1, 1, 3, 4, 4, 0, 0, 0]  Y=[1, 4, 4, 1, 3, 0, 0, 0]
[5: (5, 3), 5: (5, 4), 5: (5, 5)]

X=[1, 1, 3, 4, 4, 5, 0, 0]  Y=[1, 4, 4, 1, 3, 3, 0, 0]
[6: (5, 4), 6: (5, 5)]

X=[1, 1, 3, 4, 4, 5, 5, 0]  Y=[1, 4, 4, 1, 3, 3, 4, 0]
[7: (5, 5)]

X=[1, 1, 3, 4, 4, 5, 5, 5]  Y=[1, 4, 4, 1, 3, 3, 4, 5]


In [74]:
# Ejemplo del resultado al que llega el algoritmo
initial_state = Estado()
final_state = depthFirstSearch(initial_state)
print(final_state)

X=[3, 4, 2, 1, 2, 1, 1, 1]  Y=[3, 1, 1, 4, 3, 3, 2, 1]


In [ ]:
# [3,2,2,2,1,1,1,1]
#X = [](1, 1, 3, 4, 4, 5, 5, 5)
#Y = [](1, 4, 4, 1, 3, 3, 4, 5)

# i = 0 x=1 y=1
#X = [](1, 0, 0, 0, 0, 0, 0, 0)
#Y = [](1, 0, 0, 0, 0, 0, 0, 0)

## ESTADO 0
# i = 0 x=0 y=0
#X = [](0, 0, 0, 0, 0, 0, 0, 0)
#Y = [](0, 0, 0, 0, 0, 0, 0, 0)